# Regression MPG With Analytics Zoo

Here we are going to demonstrate an Analytics Zoo example with the Keras-style API.

In a regression problem, we aim to predict the output of a continuous value, like a price or a probability. Contrast this with a classification problem, where we aim to select a class from a list of classes (for example, where a picture contains an apple or an orange, recognizing which fruit is in the picture).

This notebook uses the classic Auto MPG Dataset and builds a model to predict the fuel efficiency of late-1970s and early 1980s automobiles. To do this, we'll provide the model with a description of many automobiles from that time period. This description includes attributes like: cylinders, displacement, horsepower, and weight.

This example uses the Analytics Zoo Keras-style API.

## Intialization

* import necesary libraries

In [ ]:
import pandas as pd

In [ ]:
import os

from zoo.common.nncontext import *
sc = init_nncontext("MPG Regression")
import pandas as pd
import numpy as np

import matplotlib
matplotlib.use('Agg')
%pylab inline
import seaborn
import matplotlib.dates as md
from matplotlib import pyplot as plt

from sklearn import preprocessing

* import necessary modules

In [ ]:
from zoo.pipeline.api.keras.layers import Dense, Dropout
from zoo.pipeline.api.keras.models import Sequential

## Data Check

# About the Data

The Auto MPG dataset
The dataset is available from the UCI Machine Learning Repository.

## Get the data

First download the dataset.

In [ ]:
dataset_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin'] 
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset = dataset.dropna()
dataset.tail()

In [ ]:
origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [ ]:
x_train = train_dataset.values
y_train = train_labels.values
x_test = test_dataset.values
y_test = test_labels.values

## Data Preparation

In [ ]:

# see the shape
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print("x_test", x_test.shape)
print("y_test", y_test.shape)

## Build Model

We are going to build a Neural Network using the Keras style API:

 * Dense Layer (64 Neurons), with Dropout
 * Dense Layer (64) Neurons), With Dropout
 * Output Layer : (1 Neuron)
 


In [ ]:
def build_model(train_x):
  model = Sequential()

  model.add(Dense(64, input_dim=9))
  model.add(Dropout(0.2))
  model.add(Dense(64))
  model.add(Dropout(0.2))
  model.add(Dense(output_dim=1))
  
  model.compile(loss='mse', optimizer='rmsprop')
  return model

model = build_model(x_train)

## Train the model

In [11]:
%%time
# Train the model
print("Training begins.")
model.fit(
    x_train,
    y_train,
    batch_size=20,
    nb_epoch=20)
print("Training completed.")

Training begins.
Training completed.
CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 10.4 s


## Prediction

* BigDL models make inferences based on the given data using model.predict(val_rdd) API. A result of RDD is returned. predict_class returns the predicted points. 

In [12]:
predictions = model.predict(x_test)

In [13]:
# create the list of difference between prediction and test data
diff=[]
ratio=[]
predictions = model.predict(x_test)
p = predictions.collect()
predictions_list=[]
for u in range(len(y_test)):
    pr = p[u][0]
    predictions_list.append(pr)
    ratio.append((y_test[u]/pr)-1)
    diff.append(abs(y_test[u]- pr))

In [14]:
RMSE = np.mean(np.sqrt(np.square(diff)))
print ("RMSE = " + str(RMSE))

RMSE = 9.133555566347562
